In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install chardet

In [ ]:
import chardet

# Detect the encoding of the file
with open('/content/drive/MyDrive/Multicampus Semi-Project/data/total_rank.csv', 'rb') as f:
    encoding = chardet.detect(f.read())['encoding']

# Read the CSV file with the detected encoding
df = pd.read_csv('/content/drive/MyDrive/Multicampus Semi-Project/data/total_rank.csv', encoding=encoding)
df = df.iloc[:, 0:10]
df.rename(columns={'서울시 주거 시설 중 주택 비율': '주택 중 아파트를 제외한 건물 비율'}, inplace=True)
df.head(3)

,자치구,비상소화장치 설치개수,주택 중 아파트를 제외한 건물 비율,인구밀도(명/km^2),노후 주택 수,소방관 1명당 담당인구,화재발생건수,안전센터 1개소당 담당인구,출동소요시간,고령자 수
0,강남구,65,65.983764,13932,24073,1454,76,88184,377.153203,86606
1,강동구,87,63.813746,18841,20483,1534,51,76678,326.475806,82984
2,강북구,106,63.805248,12414,39806,1219,62,73415,338.432836,68617


In [ ]:
import plotly.express as px

columns = df.columns[1:]

for column in columns:
    df_sorted = df.sort_values(by=column)

    fig = px.bar(df_sorted, y='자치구', x=column,
                 labels={column, '자치구'},
                 title=f'{column}', orientation='h',
                 color=column, color_continuous_scale=px.colors.sequential.Reds)

    fig.update_layout(plot_bgcolor='rgba(240, 240, 240, 0.6)')

    fig.update_yaxes(tickmode='array', tickvals=df_sorted['자치구'])

    fig.show()

In [ ]:
import plotly.express as px

fig = px.pie(df, values='비상소화장치 설치개수', names='자치구', title='비상소화장치 설치개수', color_discrete_sequence=px.colors.sequential.Reds)

fig.show()

In [ ]:
import plotly.express as px

columns = df.columns[1:]

for column in columns:
  fig = px.pie(df, values=column, names='자치구', hole=0.3, title=f'{column}',
             color_discrete_sequence=px.colors.sequential.Reds)

  fig.show()

In [ ]:
import geopandas as gpd
import pandas as pd

DATA_PATH = '/content/drive/MyDrive/Multicampus Semi-Project/data/'

# 비상소화장치 개수가 포함된 데이터셋 로드
data = pd.read_csv(DATA_PATH + 'total_rank.csv')

# 서울 구별 경계를 포함한 GeoDataFrame 로드
seoul_districts = gpd.read_file(DATA_PATH + '구경계_geo.shp')

merged_data = seoul_districts.merge(data, left_on='구', right_on='자치구')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 0: invalid start byte

In [ ]:
import folium

columns = merged_data.columns[3:12]

for column in columns:
  # 서울 중심부를 중심으로 한 맵 초기화
  seoul_map = folium.Map(location=[37.5642135, 127.0016985], zoom_start=10)

  # 코로플레스 레이어 추가
  folium.Choropleth(
      geo_data=merged_data,
      name='choropleth',
      data=merged_data,
      columns=['자치구', column],
      key_on='feature.properties.자치구',
      fill_color='Reds',
      fill_opacity=0.8,
      line_opacity=0.2,
      legend_name=f'{column}'
  ).add_to(seoul_map)

  # GeoJsonTooltip을 사용하여 각 구에 마우스 오버시 구 이름을 표시하는 GeoJson 레이어 추가
  folium.GeoJson(
      merged_data,
      name='구 이름',
      style_function=lambda x: {"weight":0.5, "color":"#555555", "fillColor":"transparent"},  # 스타일 지정
      tooltip=folium.GeoJsonTooltip(
          fields=['자치구'],
          aliases=['자치구: '],
          localize=True,
          sticky=False,
          labels=True,
          style="""
              background-color: #F0EFEF;
              border: 2px solid black;
              border-radius: 3px;
              box-shadow: 3px;
          """,
          max_width=250,
      )
  ).add_to(seoul_map)

  # 맵 표시
  #seoul_map.save('seoul_map.html')

  display(seoul_map)

In [ ]:
import plotly.express as px

column = '비상소화장치 설치개수'

df_sorted = df.sort_values(by=column)

bottom_5_values = df_sorted.nsmallest(5, column)[['자치구', column]]

fig = px.bar(bottom_5_values, y='자치구', x=column,
              labels={column, '자치구'},
              title=f'{column} 하위 5개구', orientation='h',
              color=column, color_continuous_scale=px.colors.sequential.Reds)

fig.update_layout(plot_bgcolor='rgba(240, 240, 240, 0.6)')

fig.update_yaxes(tickmode='array', tickvals=bottom_5_values['자치구'])

fig.show()

In [ ]:
import plotly.express as px

columns = df.columns[2:]

for column in columns:
    df_sorted = df.sort_values(by=column, ascending=False)

    top_5_values = df_sorted.head(5)[['자치구', column]]

    top_5_values = top_5_values.iloc[::-1]

    fig = px.bar(top_5_values, y='자치구', x=column,
                 labels={column, '자치구'},
                 title=f'{column} 상위 5개구', orientation='h',
                 color=column, color_continuous_scale=px.colors.sequential.Reds)

    fig.update_layout(plot_bgcolor='rgba(240, 240, 240, 0.6)')

    fig.update_yaxes(tickmode='array', tickvals=top_5_values['자치구'])

    fig.show()